In [1]:
import pathlib
import contextlib
import re

import requests
import pandas as pd

In [2]:
files_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/GetFiles?countryCode={country_code}'
download_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName={filename}'

headers = {"Accept": "application/json", 'User-Agent': 'test-agent'}
data_dir = pathlib.Path('~/data/euris').expanduser()
                    
# Rhein (CH, DE), Rhin (FR), Canal d’Alsace (F) Rijn (NL), Waal (NL), Boven Merwede (NL), Beneden Merwede (NL), Noord (NL), Nieuwe Maas (NL), Nieuwe Waterweg (NL), Oude Maas (NL), Hartelkanaal (NL), Pannerdensch Kanaal (NL), Nederrijn (NL), Lek (NL)
# Donau (DE), Dunaj (SK), Duna (HU), Dunav (HR), Дунав (RS), Dunărea (RO), Дунав (BG), Дунай (UA)
# Canal de Bourbourg, Canal de la Colme, Aa, Canal d’Aire, Canal de la Deule, Canal de la Sensee (FR), Lys / Leie (FR, BE), Escaut / Scheldt / Schelde (FR, BE)
# Labe (CS), Elbe (DE) Vltava (CS), Moldau (DE) Weser (DE), Vezera (CS)
# Moselle (FR), Musel (LU), Mosel (DE)

countries = ['NL', 'BE', 'DE', 'FR', 'CH', 'BG', 'UA', 'HU', 'HR', 'SK', 'RO', 'CS', 'LU', 'AT', 'GE']

In [3]:
resp = requests.get(files_url.format(country_code='LU'), headers={"Accept": "application/json", 'User-Agent': 'test-agent'})
resp

<Response [200]>

In [4]:
path_re = re.compile(r'(?P<country>[A-Z]{2})_(?P<dataset>[\w]+)_(?P<date>[\d]+)_(?P<version>v\d+\.\d+)\.zip')


data = resp.json()
dfs = []
for country_code in countries:
    url = files_url.format(country_code=country_code)
    resp = requests.get(url, headers=headers)
    data = resp.json()
    for row in data:
        name = row['name']
        match = path_re.search(name)
        if match:
            row.update(match.groupdict())
    df_i = pd.DataFrame(data)
    dfs.append(df_i) 

files = pd.concat(dfs)
set(files['name'])

files


,name,countryCode,lastModified,fileSizeInB,country,dataset,date,version
0,NL_NetworkData_20230927_v2.4.zip,NL,2023-09-27T11:46:37+02:00,3384784,NL,NetworkData,20230927,v2.4
1,NL_NetworkData_20231011_v2.4.zip,NL,2023-10-12T00:00:19+02:00,3817798,NL,NetworkData,20231011,v2.4
2,NL_NetworkData_20231025_v2.4.zip,NL,2023-10-26T00:00:33+02:00,3793950,NL,NetworkData,20231025,v2.4
3,NL_NetworkData_20240417_v2.4.zip,NL,2024-04-18T00:00:09+02:00,2220,NL,NetworkData,20240417,v2.4
4,NL_NetworkData_20240429_v2.4.zip,NL,2024-04-29T10:54:50+02:00,3765799,NL,NetworkData,20240429,v2.4
...,...,...,...,...,...,...,...,...
11,AT_RisIndex_20230927_v3.0.zip,AT,2023-09-27T11:51:33+02:00,151025,AT,RisIndex,20230927,v3.0
12,AT_RisIndex_20240423_v3.0.zip,AT,2024-04-24T00:00:09+02:00,125792,AT,RisIndex,20240423,v3.0
13,AT_RisIndex_20240429_v3.0.zip,AT,2024-04-29T11:06:30+02:00,125790,AT,RisIndex,20240429,v3.0
14,AT_RisIndex_20240527_v3.0.zip,AT,2024-05-27T11:00:18+02:00,149423,AT,RisIndex,20240527,v3.0


In [5]:
last_files = files.sort_values(['countryCode', 'dataset', 'lastModified']).groupby(['countryCode', 'dataset']).last().reset_index()
last_files

,countryCode,dataset,name,lastModified,fileSizeInB,country,date,version
0,AT,NetworkData,AT_NetworkData_20250512_v2.4.zip,2025-05-13T00:01:25+02:00,219487,AT,20250512,v2.4
1,AT,RisIndex,AT_RisIndex_20250512_v3.0.zip,2025-05-13T00:02:04+02:00,181130,AT,20250512,v3.0
2,BE,NetworkData,BE_NetworkData_20250521_v2.4.zip,2025-05-22T00:01:57+02:00,1908652,BE,20250521,v2.4
3,BE,RisIndex,BE_RisIndex_20250210_v3.0.zip,2025-02-11T00:01:00+01:00,1030784,BE,20250210,v3.0
4,BG,NetworkData,BG_NetworkData_20250225_v2.4.zip,2025-02-26T00:00:30+01:00,65433,BG,20250225,v2.4
5,BG,RisIndex,BG_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,140174,BG,20240527,v3.0
6,DE,NetworkData,DE_NetworkData_20250521_v2.4.zip,2025-05-22T00:01:57+02:00,2296710,DE,20250521,v2.4
7,DE,RisIndex,DE_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,2282383,DE,20240527,v3.0
8,FR,NetworkData,FR_NetworkData_20241106_v2.4.zip,2024-11-07T00:00:37+01:00,3187073,FR,20241106,v2.4
9,FR,RisIndex,FR_RisIndex_20241106_v3.0.zip,2024-11-07T00:04:13+01:00,3742907,FR,20241106,v3.0


In [6]:
for i, row in last_files.iterrows():
    url = download_url.format(filename=row['name'])
    with contextlib.chdir(data_dir):
        !curl -OJ '{url}' 
        !unzip -o '{row['name']}'
    

curl: option -C: --continue-at and --remote-header-name cannot be combined
curl: try 'curl --help' or 'curl --manual' for more information
Archive:  AT_NetworkData_20250512_v2.4.zip
  inflating: BerthArea_AT_20250512.geojson  
  inflating: Berth_AT_20250512.geojson  
  inflating: BridgeArea_AT_20250512.geojson  
  inflating: BridgeOpening_AT_20250512.geojson  
  inflating: FairwaySection_AT_20250512.geojson  
  inflating: LockChamberArea_AT_20250512.geojson  
  inflating: LockChamber_AT_20250512.geojson  
  inflating: LockComplex_AT_20250512.geojson  
  inflating: Node_AT_20250512.geojson  
  inflating: PassageLine_AT_20250512.geojson  
  inflating: SailingSpeed_AT_20250512.geojson  
  inflating: Terminal_AT_20250512.geojson  
curl: option -C: --continue-at and --remote-header-name cannot be combined
curl: try 'curl --help' or 'curl --manual' for more information
Archive:  AT_RisIndex_20250512_v3.0.zip
  inflating: RisIndex_AT_20250512.xlsx  
curl: option -C: --continue-at and --remote

In [7]:
!ls {data_dir}

AT_NetworkData_20250512_v2.4.zip        LockChamber_SK_20250512.geojson
AT_RisIndex_20250512_v3.0.zip           LockChamberArea_AT_20250512.geojson
BE_NetworkData_20250521_v2.4.zip        LockChamberArea_BE_20250521.geojson
BE_RisIndex_20250210_v3.0.zip           LockChamberArea_DE_20250521.geojson
Berth_AT_20250512.geojson               LockChamberArea_FR_20241106.geojson
Berth_BE_20250521.geojson               LockChamberArea_LU_20240527.geojson
Berth_BG_20250225.geojson               LockChamberArea_NL_20250521.geojson
Berth_DE_20250521.geojson               LockChamberArea_RO_20250225.geojson
Berth_FR_20241106.geojson               LockChamberArea_SK_20250512.geojson
Berth_HR_20240527.geojson               LockChamberPart_BE_20250521.geojson
Berth_HU_20241118.geojson               LockChamberPartArea_BE_20250521.geojson
Berth_LU_20240527.geojson               LockComplex_AT_20250512.geojson
Berth_NL_20250521.geojson               LockComplex_BE_20250521.geojson
Berth_RO_20250225.ge

In [8]:
!open {data_dir}